In [2]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from skimage import io, transform, color
from skimage.feature import hog
import glob     
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image
import os
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
import cv2
import os
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm
import cv2
import shutil

In [3]:
train  = pd.read_csv('trainset.csv')
test   = pd.read_csv('testset.csv')

In [10]:
imgs= os.listdir('Tumor\\images\\images')
id_train = dict(zip(train['Id'], train['Class']))

In [ ]:
ids

In [11]:
# create a list for both images and labels
images_train = []
labels_train = []

In [ ]:
for index, row in train.iterrows():
  im_id = row['Id']
  if im_id in id_train :
    label = id_train [im_id]
    images_train = os.path.join(imgs, f"{im_id}.png")
    if os.path.exists(image_path):  # Check if the image file exists
        image = cv2.imread(image_path)  # Load the image using cv2

        if image is not None:  # Check if the image was loaded successfully
            images_train.append(image)
            labels_train.append(label)

train_data= np.array(images_train)
train_label = np.array(labels_train)

In [10]:
# Get the image IDs from the test CSV
test_id = list(test['Id'])

In [11]:
image_test = []
for index, row in test.iterrows():
  image_id = row['Id']
  if image_id in test_id:
    image_path = os.path.join(image_folder, f"{image_id}.png")
    # Check if the image file exists
    if os.path.exists(image_path):
      # Load the image using cv2
      image = cv2.imread(image_path)
      image_test.append(image)

test_data = np.array(image_test)

<ipython-input-11-b37ddeeeb83b>:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_data = np.array(image_test)


# resize the images

In [12]:
# train images

target_size = (200, 200)  # Replace with your desired target size

# Initialize an empty list to store resized images
resized_images = []

# Loop through each image in X_train and resize
for image in train_data:
    resized_image = cv2.resize(image, target_size)
    resized_images.append(resized_image)

# Convert the list of resized images back to a NumPy array
resized_train = np.array(resized_images)

In [13]:
# test images

target_size = (200, 200)  # Replace with your desired target size

# Initialize an empty list to store resized images
resized_images = []

# Loop through each image in X_train and resize
for image in test_data:
    resized_image = cv2.resize(image, target_size)
    resized_images.append(resized_image)

# Convert the list of resized images back to a NumPy array
resized_test = np.array(resized_images)

In [14]:
resized_train.shape, resized_test.shape

((539, 200, 200, 3), (229, 200, 200, 3))

# Normalize images

In [15]:
resized_train = resized_train/255

resized_test = resized_test/255

In [16]:
train_flatten = resized_train.reshape(resized_train.shape[0], -1)
test_flatten = resized_test.reshape(resized_test.shape[0], -1)

In [17]:
train_flatten.shape, test_flatten.shape

((539, 120000), (229, 120000))

In [18]:
# Convert feature list to numpy array
train_features = np.array(train_flatten)

# Convert feature list to numpy array
test_features = np.array(test_flatten)

In [37]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42,k_neighbors=10,  sampling_strategy='auto')
smote.fit(train_features, train_label)
X_resampled, y_resampled = smote.fit_resample(train_features, train_label)


In [38]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled  , test_size=0.2, random_state=42)

In [39]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((720, 120000), (180, 120000), (720,), (180,))

In [40]:
pca = PCA(n_components=150 )  # You can adjust the number of components
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)


In [29]:
X_train_pca.shape, X_test_pca.shape

((720, 150), (180, 150))

# **Modeling**

In [30]:
svm_classifier = SVC(kernel='rbf', C=3, gamma=0.001)
svm_classifier.fit(X_train_pca, y_train)

# Make predictions
y_pred1 = svm_classifier.predict(X_train_pca)
y_pred2 = svm_classifier.predict(X_test_pca)


accuracy2 = accuracy_score(y_test, y_pred2)


print("Accuracy2:", accuracy2)

# Calculate confusion matrix
conf_matrix2 = confusion_matrix(y_test, y_pred2)
print("Confusion Matrix1:\n", conf_matrix2)

# Generate classification report
class_report2 = classification_report(y_test, y_pred2)
print("Classification Report:\n", class_report2)

Accuracy2: 0.8777777777777778
Confusion Matrix1:
 [[49  2  3]
 [12 53  0]
 [ 4  1 56]]
Classification Report:
               precision    recall  f1-score   support

      Benign       0.75      0.91      0.82        54
   Malignant       0.95      0.82      0.88        65
      Normal       0.95      0.92      0.93        61

    accuracy                           0.88       180
   macro avg       0.88      0.88      0.88       180
weighted avg       0.89      0.88      0.88       180



In [31]:
X_test_pca = pca.transform(test_features)
predictions = svm_classifier.predict(X_test_pca)

In [32]:
test['labels'] =predictions
test['labels'].value_counts()

Benign       156
Malignant     46
Normal        27
Name: labels, dtype: int64

In [33]:
submit = pd.read_csv('/content/sample_submition4.csv')
submit

,Id,Class
0,8560,A
1,7920,A
2,8993,A
3,8126,A
4,9912,A
...,...,...
224,8343,A
225,1255,A
226,4573,A
227,3919,A


In [34]:
submit['Id'] = test['Id']
submit['Class'] =predictions # our model predictions on the test dataset
submit

,Id,Class
0,8560,Benign
1,7920,Benign
2,8993,Malignant
3,8126,Normal
4,9912,Benign
...,...,...
224,8343,Malignant
225,1255,Benign
226,4573,Benign
227,3919,Benign


In [35]:
submit.to_csv('sample_submission_fatima5.csv', index=False)

# **Other Models**

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier



# Create a k-NN classifier
knn_classifier = KNeighborsClassifier(n_neighbors=2)  # You can adjust the number of neighbors

# Assuming you have X_train_pca, y_train, X_val_pca, y_val
# Train the k-NN classifier
knn_classifier.fit(X_train_pca, y_train)

# Make predictions
y_pred1 = knn_classifier.predict(X_train_pca)
y_pred2 = knn_classifier.predict(X_val_pca)

# Calculate accuracy
accuracy1 = accuracy_score(y_train, y_pred1)
accuracy2 = accuracy_score(y_val, y_pred2)
print("Accuracy1:", accuracy1)
print("Accuracy2:", accuracy2)

# Calculate confusion matrix
conf_matrix1 = confusion_matrix(y_train, y_pred1)
conf_matrix2 = confusion_matrix(y_val, y_pred2)
print("Confusion Matrix1:\n", conf_matrix1)
print("Confusion Matrix2:\n", conf_matrix2)

# Generate classification report
class_report1 = classification_report(y_train, y_pred1)
class_report2 = classification_report(y_val, y_pred2)
print("Classification Report1:\n", class_report1)
print("Classification Report2:\n", class_report2)


Accuracy1: 0.8955916473317865
Accuracy2: 0.5925925925925926
Confusion Matrix1:
 [[243   0   0]
 [ 19  99   0]
 [ 17   9  44]]
Confusion Matrix2:
 [[45  7  5]
 [13 13  4]
 [10  5  6]]
Classification Report1:
               precision    recall  f1-score   support

      Benign       0.87      1.00      0.93       243
   Malignant       0.92      0.84      0.88       118
      Normal       1.00      0.63      0.77        70

    accuracy                           0.90       431
   macro avg       0.93      0.82      0.86       431
weighted avg       0.90      0.90      0.89       431

Classification Report2:
               precision    recall  f1-score   support

      Benign       0.66      0.79      0.72        57
   Malignant       0.52      0.43      0.47        30
      Normal       0.40      0.29      0.33        21

    accuracy                           0.59       108
   macro avg       0.53      0.50      0.51       108
weighted avg       0.57      0.59      0.58       108



## DT

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Create a Decision Tree classifier
decision_tree_classifier = DecisionTreeClassifier(
                                                   random_state=14,
                                                )  # You can adjust random_state if needed

# Assuming you have X_train_pca, y_train, X_val_pca, y_val
# Train the Decision Tree classifier
decision_tree_classifier.fit(X_train_pca, y_train)

# Make predictions
y_pred1 = decision_tree_classifier.predict(X_train_pca)
y_pred2 = decision_tree_classifier.predict(X_val_pca)

# Calculate accuracy
accuracy1 = accuracy_score(y_train, y_pred1)
accuracy2 = accuracy_score(y_val, y_pred2)
print("Accuracy1:", accuracy1)
print("Accuracy2:", accuracy2)

# Calculate confusion matrix
conf_matrix1 = confusion_matrix(y_train, y_pred1)
conf_matrix2 = confusion_matrix(y_val, y_pred2)
print("Confusion Matrix1:\n", conf_matrix1)
print("Confusion Matrix2:\n", conf_matrix2)

# Generate classification report
class_report1 = classification_report(y_train, y_pred1)
class_report2 = classification_report(y_val, y_pred2)
print("Classification Report1:\n", class_report1)
print("Classification Report2:\n", class_report2)


Accuracy1: 1.0
Accuracy2: 0.4444444444444444
Confusion Matrix1:
 [[243   0   0]
 [  0 118   0]
 [  0   0  70]]
Confusion Matrix2:
 [[38 14  5]
 [19  8  3]
 [17  2  2]]
Classification Report1:
               precision    recall  f1-score   support

      Benign       1.00      1.00      1.00       243
   Malignant       1.00      1.00      1.00       118
      Normal       1.00      1.00      1.00        70

    accuracy                           1.00       431
   macro avg       1.00      1.00      1.00       431
weighted avg       1.00      1.00      1.00       431

Classification Report2:
               precision    recall  f1-score   support

      Benign       0.51      0.67      0.58        57
   Malignant       0.33      0.27      0.30        30
      Normal       0.20      0.10      0.13        21

    accuracy                           0.44       108
   macro avg       0.35      0.34      0.34       108
weighted avg       0.40      0.44      0.41       108



# RF

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Create a Random Forest classifier
random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=100,max_depth=20,
                                                  min_samples_split=2, min_samples_leaf=1,
                                                  min_weight_fraction_leaf=0.0, max_features='sqrt',
                                                  max_leaf_nodes=None, min_impurity_decrease=0.0,
                                                  bootstrap=True, oob_score=False, n_jobs=None,
                                                  verbose=0, warm_start=False,
                                                  class_weight={'Normal':4, 'Malignant':2}, ccp_alpha=0.0, max_samples=0.5)
# You can adjust n_estimators and other hyperparameters

# Assuming you have X_train_pca, y_train, X_val_pca, y_val
# Train the Random Forest classifier
random_forest_classifier.fit(X_train_pca, y_train)

# Make predictions
y_pred1 = random_forest_classifier.predict(X_train_pca)
y_pred2 = random_forest_classifier.predict(X_val_pca)

# Calculate accuracy
accuracy1 = accuracy_score(y_train, y_pred1)
accuracy2 = accuracy_score(y_val, y_pred2)
print("Accuracy1:", accuracy1)
print("Accuracy2:", accuracy2)

# Calculate confusion matrix
conf_matrix1 = confusion_matrix(y_train, y_pred1)
conf_matrix2 = confusion_matrix(y_val, y_pred2)
print("Confusion Matrix1:\n", conf_matrix1)
print("Confusion Matrix2:\n", conf_matrix2)

# Generate classification report
class_report1 = classification_report(y_train, y_pred1)
class_report2 = classification_report(y_val, y_pred2)
print("Classification Report1:\n", class_report1)
print("Classification Report2:\n", class_report2)


Accuracy1: 0.9976798143851509
Accuracy2: 0.5370370370370371
Confusion Matrix1:
 [[243   0   0]
 [  1 117   0]
 [  0   0  70]]
Confusion Matrix2:
 [[57  0  0]
 [30  0  0]
 [20  0  1]]
Classification Report1:
               precision    recall  f1-score   support

      Benign       1.00      1.00      1.00       243
   Malignant       1.00      0.99      1.00       118
      Normal       1.00      1.00      1.00        70

    accuracy                           1.00       431
   macro avg       1.00      1.00      1.00       431
weighted avg       1.00      1.00      1.00       431

Classification Report2:
               precision    recall  f1-score   support

      Benign       0.53      1.00      0.70        57
   Malignant       0.00      0.00      0.00        30
      Normal       1.00      0.05      0.09        21

    accuracy                           0.54       108
   macro avg       0.51      0.35      0.26       108
weighted avg       0.48      0.54      0.38       108



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Extra Tree

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import numpy as np

# Create an Extra Trees classifier
extra_trees_classifier = ExtraTreesClassifier(n_estimators=100, random_state=42)
# You can adjust n_estimators and other hyperparameters

# Assuming you have X_train_pca, y_train, X_val_pca, y_val
# Train the Extra Trees classifier
extra_trees_classifier.fit(X_train_pca, y_train)

# Make predictions
y_pred1 = extra_trees_classifier.predict(X_train_pca)
y_pred2 = extra_trees_classifier.predict(X_val_pca)

# Calculate accuracy
accuracy1 = accuracy_score(y_train, y_pred1)
accuracy2 = accuracy_score(y_val, y_pred2)
print("Accuracy1:", accuracy1)
print("Accuracy2:", accuracy2)

# Calculate confusion matrix
conf_matrix1 = confusion_matrix(y_train, y_pred1)
conf_matrix2 = confusion_matrix(y_val, y_pred2)
print("Confusion Matrix1:\n", conf_matrix1)
print("Confusion Matrix2:\n", conf_matrix2)

# Generate classification report
class_report1 = classification_report(y_train, y_pred1)
class_report2 = classification_report(y_val, y_pred2)
print("Classification Report1:\n", class_report1)
print("Classification Report2:\n", class_report2)


Accuracy1: 1.0
Accuracy2: 0.5370370370370371
Confusion Matrix1:
 [[243   0   0]
 [  0 118   0]
 [  0   0  70]]
Confusion Matrix2:
 [[56  0  1]
 [30  0  0]
 [19  0  2]]
Classification Report1:
               precision    recall  f1-score   support

      Benign       1.00      1.00      1.00       243
   Malignant       1.00      1.00      1.00       118
      Normal       1.00      1.00      1.00        70

    accuracy                           1.00       431
   macro avg       1.00      1.00      1.00       431
weighted avg       1.00      1.00      1.00       431

Classification Report2:
               precision    recall  f1-score   support

      Benign       0.53      0.98      0.69        57
   Malignant       0.00      0.00      0.00        30
      Normal       0.67      0.10      0.17        21

    accuracy                           0.54       108
   macro avg       0.40      0.36      0.29       108
weighted avg       0.41      0.54      0.40       108



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
